In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("MovieGenre.csv")
print(df.shape)
df.head(5)


(40108, 6)


,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [3]:
#ensure everything is unique except for score and genre
print(df['imdbId'].is_unique)
print(len(df['imdbId']) - df['imdbId'].nunique())

print(df['Imdb Link'].is_unique)
print(len(df['Imdb Link']) - df['Imdb Link'].nunique())

print(df['Title'].is_unique)
print(len(df['Title']) - df['Title'].nunique())

print(df['Poster'].is_unique)
print(len(df['Poster']) - df['Poster'].nunique())

False
593
False
593
False
638
False
1454


In [4]:
#remove not unique values
df = df.drop_duplicates(subset=['imdbId'], keep=False)
df = df.drop_duplicates(subset=['Imdb Link'], keep=False)
df = df.drop_duplicates(subset=['Title'], keep=False)
df = df.drop_duplicates(subset=['Poster'], keep=False)

In [5]:
print(df['imdbId'].is_unique)
print(len(df['imdbId']) - df['imdbId'].nunique())

print(df['Imdb Link'].is_unique)
print(len(df['Imdb Link']) - df['Imdb Link'].nunique())

print(df['Title'].is_unique)
print(len(df['Title']) - df['Title'].nunique())

print(df['Poster'].is_unique)
print(len(df['Poster']) - df['Poster'].nunique())

True
0
True
0
True
0
True
0


In [6]:
#Check null values
df.isnull().sum()

imdbId          0
Imdb Link       0
Title           0
IMDB Score     17
Genre         119
Poster          0
dtype: int64

In [7]:
df = df.dropna(subset=['Genre'])
df.isnull().sum()

imdbId         0
Imdb Link      0
Title          0
IMDB Score    17
Genre          0
Poster         0
dtype: int64

In [8]:
print(df.shape)

(37786, 6)


In [9]:
# split genre and turn them into a list
df['Genre'] = df['Genre'].str.split('|')

In [10]:
#Binary encoding using MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

# Fit and transform the Genre column
genre_encoded = mlb.fit_transform(df['Genre'])

# Create a DataFrame with the encoded genres
encoded_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

df = pd.concat([df.reset_index(drop=True), encoded_df], axis=1)

In [11]:
#drop irrelevant columns
df = df.drop(columns=['Genre', 'Imdb Link', 'IMDB Score'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37786 entries, 0 to 37785
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   imdbId       37786 non-null  int64 
 1   Title        37786 non-null  object
 2   Poster       37786 non-null  object
 3   Action       37786 non-null  int64 
 4   Adult        37786 non-null  int64 
 5   Adventure    37786 non-null  int64 
 6   Animation    37786 non-null  int64 
 7   Biography    37786 non-null  int64 
 8   Comedy       37786 non-null  int64 
 9   Crime        37786 non-null  int64 
 10  Documentary  37786 non-null  int64 
 11  Drama        37786 non-null  int64 
 12  Family       37786 non-null  int64 
 13  Fantasy      37786 non-null  int64 
 14  Film-Noir    37786 non-null  int64 
 15  Game-Show    37786 non-null  int64 
 16  History      37786 non-null  int64 
 17  Horror       37786 non-null  int64 
 18  Music        37786 non-null  int64 
 19  Musical      37786 non-nu

In [13]:
genre_counts = df.iloc[:, 3:].sum(axis=0)
genre_counts = genre_counts.sort_values(ascending=False)

# Print the sorted sum for each binarized genre
print(genre_counts)
print(df.shape)

Drama          19068
Comedy         12105
Romance         5919
Action          5119
Crime           5023
Thriller        4632
Horror          3853
Adventure       3655
Documentary     3411
Mystery         2296
Family          2032
Fantasy         1933
Sci-Fi          1926
Biography       1877
Animation       1635
History         1335
Music           1212
War             1109
Short            845
Musical          813
Western          810
Sport            661
Film-Noir        382
News              77
Adult              9
Talk-Show          6
Reality-TV         2
Game-Show          1
dtype: int64
(37786, 31)


In [14]:
#Drop low frequency genres
genres_to_drop = ["Adult", "Talk-Show", "Reality-TV", "Game-Show", "News", "Film-Noir", "Sport", "Western", "Musical", "Short", "War", "Music", "History"]
df = df.drop(columns=genres_to_drop)

In [15]:
#Drop rows without any remaining genres
df = df[df.iloc[:, 3:].any(axis=1)]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37316 entries, 0 to 37785
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   imdbId       37316 non-null  int64 
 1   Title        37316 non-null  object
 2   Poster       37316 non-null  object
 3   Action       37316 non-null  int64 
 4   Adventure    37316 non-null  int64 
 5   Animation    37316 non-null  int64 
 6   Biography    37316 non-null  int64 
 7   Comedy       37316 non-null  int64 
 8   Crime        37316 non-null  int64 
 9   Documentary  37316 non-null  int64 
 10  Drama        37316 non-null  int64 
 11  Family       37316 non-null  int64 
 12  Fantasy      37316 non-null  int64 
 13  Horror       37316 non-null  int64 
 14  Mystery      37316 non-null  int64 
 15  Romance      37316 non-null  int64 
 16  Sci-Fi       37316 non-null  int64 
 17  Thriller     37316 non-null  int64 
dtypes: int64(16), object(2)
memory usage: 5.4+ MB


In [17]:
#drop rows for images unavailable based on the scraper

#IDs from failed_downloads in image_scraper.ipynb
failed_downloads = [113228, 112302, 113189, 114057, 115012, 112697, 113347, 114814, 109950, 110299, 112499, 118002, 113149, 113010, 113537, 117002, 113247, 115734, 116483, 112579, 75314, 112373, 116606, 114808, 110647, 111055, 114015, 109093, 112462, 112541, 109508, 112857, 114558, 114781, 109635, 112899, 109579, 111797, 109758, 113028, 109771, 113538, 76759, 110367, 110538, 113808, 113870, 117169, 113948, 114151, 110413, 114113, 108394, 111255, 111161, 110081, 111280, 114852, 114888, 114857, 109655, 109040, 109484, 109830, 113305, 110091, 107472, 110588, 105226, 111756, 113173, 113827, 114047, 109035, 106226, 109068, 106400, 106489, 106505, 106519, 109443, 109480, 109783, 106880, 111712, 106918, 107004, 107076, 112966, 107151, 107207, 110197, 110265, 107322, 107413, 107468, 107497, 111689, 110657, 107818, 108000, 105032, 108065, 111201, 108162, 111418, 108333, 108358, 108399, 108515, 110763, 110259, 107002, 109403, 110363, 111709, 99785, 117381, 115509, 115956, 116136, 48473, 52572, 117705, 107727, 117104, 117774, 113270, 113568, 113849, 111237, 116905, 118113, 117628, 116823, 61495, 112716, 116707, 117332, 53459, 115632, 107575, 68646, 109255, 116985, 114936, 114474, 119214, 113542, 106544, 116418, 24252, 32904, 50419, 54698, 25316, 36855, 53125, 53604, 33870, 58385, 47437, 31381, 84370, 43014, 33467, 39428, 32143, 32484, 38109, 42451, 28010, 32599, 38650, 29947, 17350, 38300, 32022, 26029, 63350, 43265, 47162, 33891, 29588, 116329, 117039, 116259, 115862, 47977, 72951, 50798, 46672, 38166, 41890, 102798, 117924, 116409, 116913, 120107, 115600, 115491, 39286, 84865, 68334, 61418, 91763, 98627, 101761, 104348, 84707, 117577, 116859, 116422, 101775, 80749, 71853, 113328, 84589, 39881, 26465, 26656, 78875, 95765, 109942, 120121, 97937, 105488, 80684, 90605, 56592, 78788, 86190, 93191, 41959, 99685, 78748, 54215, 80455, 71562, 97499, 81398, 75686, 70735, 85859, 52311, 50212, 71315, 40897, 92005, 77416, 101921, 99334, 102536, 32455, 38355, 97493, 94006, 78841, 96332, 87553, 89606, 101458, 116601, 103644, 82010, 85159, 71233, 90655, 82083, 101540, 74285, 83722, 75005, 115495, 117057, 119791, 117731, 117631, 117666, 117372, 115433, 102975, 98382, 84726, 96487, 100994, 115471, 115633, 119783, 104691, 116240, 116506, 118708, 106266, 116790, 116754, 120318, 116922, 119008, 118859, 119572, 120152, 118064, 118829, 118663, 119381, 119229, 118541, 113314, 118783, 117482, 120479, 119109, 120190, 120087, 118556, 120179, 57345, 116334, 119173, 119896, 118818, 118883, 118966, 120207, 119509, 99810, 119874, 119488, 120102, 119457, 120323, 119349, 119345, 119906, 120481, 120257, 118900, 117221, 119327, 119592, 119978, 119365, 107282, 118617, 116465, 119668, 119819, 120670, 116845, 120696, 119711, 118747, 114690, 116379, 119548, 120782, 119574, 119942, 119780, 119560, 119305, 119272, 119196, 118229, 120906, 118577, 120725, 120265, 120531, 120800, 131436, 119547, 118894, 139362, 119494, 116141, 126938, 73540, 122718, 138704, 140688, 18578, 21746, 26752, 36868, 39416, 44672, 47296, 79417, 82158, 97239, 67309, 91369, 84516, 70047, 99528, 99700, 89218, 119910, 70723, 17136, 54649, 47478, 88814, 62737, 77305, 97053, 82199, 61749, 77698, 59113, 82736, 90756, 89385, 97757, 104990, 76618, 78158, 107952, 38969, 111359, 83798, 85382, 120832, 80388, 77711, 61184, 89469, 90848, 91223, 88323, 120611, 119778, 120692, 94721, 74512, 68611, 65112, 58329, 44079, 42994, 119453, 37017, 33922, 34248, 31505, 30341, 29811, 28212, 23395, 20142, 19702, 128442, 92537, 95593, 84472, 87810, 92225, 96073, 98625, 102558, 101523, 120776, 118736, 120835, 146336, 119802, 120770, 124102, 63462, 119577, 120791, 120586, 158493, 133952, 130018, 24216, 89087, 120595, 119304, 124198, 119248, 120324, 91653, 91326, 120863, 133751, 142231, 119336, 110978, 92112, 91991, 90863, 138279, 145893, 128133, 124298, 139462, 181322, 70334, 51744, 76636, 79550, 74812, 77189, 71455, 159696, 120646, 120859, 160429, 129332, 139668, 120613, 120449, 139239, 166195, 119219, 125659, 120836, 138874, 37913, 89652, 99422, 118826, 120865, 120915, 149723, 53085, 32818, 78346, 21884, 31951, 50280, 34398, 73629, 84156, 49366, 49516, 139809, 156794, 184510, 130827, 158811, 120663, 139414, 48254, 56193, 90774, 92068, 91699, 91419, 54033, 92627, 144168, 129111, 120709, 119614, 77663, 91178, 66765, 80339, 96244, 85334, 130121, 176269, 105698, 119692, 52602, 73802, 164108, 120747, 126810, 157183, 178988, 76150, 169547, 88178, 127296, 97390, 82348, 50814, 108185, 73812, 99558, 160338, 162360, 194314, 68990, 60095, 171804, 168740, 68699, 61747, 80179, 34240, 35169, 61578, 59260, 58461, 119256, 137523, 203408, 72742, 59319, 185371, 144178, 65088, 16630, 67525, 83254, 61789, 172726, 149691, 118150, 32215, 32283, 101605, 56197, 105156, 84855, 68528, 86856, 59646, 164312, 143145, 185125, 84357, 24852, 96061, 40522, 93509, 93512, 33045, 105265, 87091, 87921, 146675, 155711, 134154, 97717, 160509, 48034, 144715, 160513, 10040, 50558, 124315, 167423, 67328, 37547, 174204, 106455, 64276, 32590, 47630, 95897, 70290, 65436, 86859, 107659, 40552, 88707, 100318, 62281, 81748, 120662, 105323, 104694, 105812, 103905, 105477, 104040, 134084, 105414, 144117, 171356, 220099, 209189, 74455, 159272, 36653, 18773, 21749, 87265, 9018, 158583, 199290, 165798, 94347, 85980, 56173, 47127, 62793, 86005, 55353, 183523, 101698, 78902, 49730, 51411, 57344, 39748, 92834, 89994, 108526, 88760, 78950, 71402, 89003, 36775, 101615, 95238, 91142, 54997, 53719, 99871, 93378, 95564, 89643, 107630, 74958, 79640, 67698, 96211, 86927, 82934, 160644, 179063, 102898, 68327, 85701, 104549, 44837, 180793, 112634, 153464, 210130, 49279, 71772, 156323, 54692, 71288, 51818, 70481, 36326, 82933, 66495, 207998, 154443, 34742, 39631, 34889, 69029, 95977, 63518, 38120, 92638, 48347, 86979, 86129, 79501, 89530, 91635, 94882, 71360, 77588, 82945, 23622, 189142, 120630, 200669, 39404, 50292, 99797, 71569, 64904, 38718, 40580, 46213, 70679, 219854, 215369, 60176, 38499, 120903, 52561, 104299, 105813, 34055, 205461, 131972, 206226, 57603, 65938, 98384, 85470, 111187, 130444, 233687, 138681, 178050, 32881, 46303, 62885, 234137, 259207, 216772, 187696, 190798, 204626, 202677, 191037, 182508, 93177, 95294, 57887, 32234, 52846, 36261, 88024, 202641, 162236, 75754, 54387, 37638, 82307, 219952, 146984, 180073, 92106, 96219, 95927, 181536, 200720, 223530, 219653, 149624, 181865, 97662, 72735, 176883, 102388, 116119, 162024, 92546, 92641, 92655, 92695, 92718, 92804, 92809, 92854, 92974, 93036, 93093, 93209, 93225, 93278, 93418, 93467, 93560, 93562, 190524, 118694, 212985, 216787, 177650, 249893, 166276, 191636, 244504, 192071, 31742, 96223, 90213, 100842, 39190, 37800, 80421, 91474, 71771, 211492, 55895, 245120, 240913, 98524, 58453, 152015, 96875, 203009, 157411, 77716, 52902, 48248, 94142, 244316, 161083, 45810, 48605, 90770, 212720, 255819, 228242, 215139, 179473, 245238, 58953, 58249, 45699, 59124, 91983, 59740, 57693, 82966, 84788, 69467, 65777, 93227, 48261, 49966, 89716, 76786, 242193, 96764, 94669, 94715, 94739, 94746, 94747, 94761, 94783, 94889, 95088, 95145, 95178, 95326, 95638, 95736, 95993, 96029, 96037, 96071, 96087, 96098, 91949, 96280, 96320, 96380, 96425, 96945, 97109, 97243, 97257, 97276, 97435, 97438, 97458, 97500, 97567, 97607, 97626, 97637, 97714, 97967, 163025, 123948, 259442, 98188, 98343, 98356, 98436, 98536, 108517, 87130, 60668, 80377, 80402, 261392, 234829, 233699, 269341, 255321, 63032, 93677, 50933, 86352, 260866, 206314, 273799, 240890, 82100, 58283, 80500, 80549, 80634, 80641, 243255, 166924, 67093, 87666, 272020, 246578, 278102, 249478, 177888, 139951, 68168, 76221, 76342, 24264, 60437, 80671, 80715, 80716, 80724, 233841, 50539, 283509, 261066, 87747, 259393, 90852, 60714, 88103, 94318, 35423, 120737, 63415, 105287, 280707, 45555, 117653, 242795, 93106, 66585, 80754, 160399, 268696, 71691, 66026, 294289, 276501, 106223, 87233, 295552, 46828, 265713, 69995, 60200, 44391, 60218, 75276, 260746, 143861, 301893, 77199, 75718, 51876, 50986, 80057, 27286, 39645, 38057, 127516, 80798, 80836, 80863, 80904, 80928, 76276, 81109, 81176, 81178, 81249, 81259, 32520, 35262, 32993, 34116, 81480, 81529, 76729, 86325, 54988, 102103, 246134, 98994, 266391, 72737, 99892, 93854, 103035, 239507, 181912, 88931, 65916, 108311, 110157, 86397, 103253, 52600, 92652, 50634, 121765, 77523, 254199, 268690, 51496, 44557, 71532, 56406, 90570, 45679, 57541, 80097, 210065, 87042, 40823, 51739, 91472, 41866, 244094, 89798, 254455, 295178, 246500, 74608, 40866, 35160, 292066, 82054, 88885, 90799, 80931, 37219, 55623, 270480, 275688, 84133, 99128, 32390, 99512, 101821, 110557, 39111, 44829, 171685, 283832, 245479, 164810, 64373, 86373, 58230, 93113, 82362, 99581, 84049, 42898, 310793, 298744, 101453, 86361, 81617, 82081, 82175, 82222, 82329, 82457, 82507, 84090, 82639, 82648, 82671, 82801, 82748, 82910, 259484, 209933, 87075, 292501, 201020, 297162, 42895, 283644, 77549, 102368, 103112, 83089, 83113, 83163, 83178, 83189, 83190, 246460, 293815, 303396, 118767, 71671, 89378, 83686, 83694, 83726, 182060, 106453, 75968, 66563, 45917, 249371, 309912, 42325, 106753, 85794, 282698, 330069, 245341, 246498, 284655, 109034, 102500, 86978, 71334, 63091, 58371, 308514, 54417, 64323, 83885, 83908, 83943, 83996, 84210, 85852, 84302, 84316, 84359, 84422, 84481, 84628, 83107, 84756, 84777, 84786, 83941, 84935, 68441, 61523, 95626, 91575, 240980, 324080, 266971, 53337, 35417, 69765, 60182, 50383, 65054, 283323, 61834, 36182, 70460, 98093, 94020, 314062, 311519, 109327, 78504, 102849, 100691, 290145, 317226, 285861, 308672, 250067, 286516, 266543, 295700, 263671, 101020, 69291, 66831, 66301, 100827, 71143, 74281, 54759, 31225, 59358, 50763, 57811, 100211, 54310, 54428, 117959, 100301, 102900, 100670, 339034, 280490, 293452, 55972, 60814, 106792, 175777, 31602, 29808, 99587, 79510, 93660, 101991, 58621, 74811, 308476, 283387, 113533, 100053, 319829, 94764, 310149, 73345, 41792, 112628, 64437, 109369, 51776, 62467, 108557, 46438, 91830, 68768, 68361, 235060, 44741, 286476, 243232, 317950, 301684, 325258, 304711, 71385, 87384, 283422, 104850, 327850, 330602, 86541, 103882, 115561, 120125, 90735, 101516, 87909, 251433, 59221, 91251, 105691, 105291, 62190, 63591, 110169, 45205, 102768, 163745, 53593, 177068, 104321, 61809, 102460, 317676, 313542, 339579, 165832, 74802, 71276, 38661, 34409, 119935, 56211, 48956, 75902, 339607, 242653, 333764, 37635, 85426, 104265, 9968, 109951, 65832, 102034, 100151, 102216, 104627, 70849, 48424, 107711, 105211, 150662, 90180, 94332, 24025, 27630, 26942, 22286, 107358, 15163, 62765, 31455, 69097, 30637, 87892, 23027, 42949, 92925, 114086, 102818, 59084, 26778, 87280, 167260, 287963, 329691, 71577, 53133, 65911, 107750, 310924, 162958, 54469, 61581, 37382, 33717, 39302, 36244, 33149, 22268, 104009, 60980, 63800, 296042, 72730, 97028, 6864, 54673, 290879, 67589, 56580, 345551, 111218, 166158, 13140, 323872, 304391, 15400, 60420, 335345, 60827, 35753, 56704, 360009, 317842, 354766, 305583, 299172, 326977, 90036, 58930, 96921, 104783, 45094, 41098, 44762, 82516, 58725, 380615, 349169, 36515, 65780, 85370, 60736, 51077, 188055, 88650, 282410, 113725, 73822, 32943, 36230, 48673, 76734, 62229, 86984, 73636, 166707, 105327, 105017, 63285, 36696, 50861, 46268, 102011, 56943, 169639, 38854, 155713, 107617, 62480, 33105, 177215, 86022, 58458, 50613, 346336, 104377, 284880, 259072, 99850, 146455, 77532, 66549, 92117, 70948, 53976, 46085, 66580, 258760, 23694, 60841, 70518, 51849, 71249, 56412, 17668, 52847, 41699, 87193, 55499, 38669, 62185, 88322, 73918, 100260, 69048, 65481, 95756, 52655, 61189, 105159, 64806, 124207, 60522, 77360, 99399, 93913, 18455, 280674, 189553, 104346, 54462, 49456, 26174, 85478, 43880, 48973, 24368, 38873, 28478, 32617, 37449, 75612, 41944, 100112, 17765, 21890, 33874, 35019, 79116, 83090, 77235, 89160, 87337, 31022, 31448, 63759, 66090, 18051, 93871, 65889, 98528, 39040, 46963, 33774, 37135, 32851, 37988, 61420, 54188, 71565, 115738, 120408, 22913, 377752, 297753, 37522, 96953, 37865, 57681, 104573, 61678, 282552, 40884, 45586, 53622, 93886, 114371, 73092, 87597, 78437, 77629, 47365, 89945, 95468, 23563, 50171, 64395, 90197, 47216, 59470, 69280, 69822, 99292, 99623, 76111, 52002, 28108, 111216, 40806, 35369, 64381, 72848, 68424, 88315, 280990, 51724, 81291, 87795, 64840, 67314, 41268, 39017, 38259, 368008, 64137, 115665, 250638, 64921, 291392, 63634, 65125, 57191, 80062, 57997, 71115, 63803, 362129, 58294, 80380, 389988, 33029, 29852, 89543, 52549, 103594, 80009, 78975, 90985, 101301, 98596, 362174, 360130, 273689, 38048, 360845, 385004, 53644, 403910, 53716, 50301, 58018, 59166, 55069, 275468, 40705, 88074, 162662, 125510, 424885, 13257, 14142, 15324, 15624, 15881, 18737, 21730, 22158, 23293, 23458, 23649, 23686, 23948, 24055, 24593, 24865, 24894, 25301, 25898, 27478, 27652, 27884, 28096, 28174, 28683, 28804, 29322, 29442, 29912, 31491, 31867, 33945, 34428, 34449, 35209, 35238, 35415, 35979, 37055, 37077, 37094, 37469, 37820, 38107, 38160, 38338, 38369, 38913, 39926, 40308, 40550, 40636, 40919, 41507, 42531, 45053, 46436, 47123, 47191, 47203, 47444, 47677, 48579, 48954, 49710, 50168, 54020, 54460, 55019, 55728, 56058, 56194, 56512, 56700, 57809, 57877, 58390, 58530, 58724, 59127, 59309, 59311, 59410, 59798, 59825, 60802, 61101, 61882, 62425, 63426, 63654, 63665, 64177, 64612, 64886, 65073, 65642, 65772, 67023, 67072, 67333, 67341, 67919, 68291, 69958, 70300, 70726, 71431, 71487, 71604, 72610, 72826, 72933, 72973, 73179, 74084, 74605, 74633, 74870, 74883, 75376, 76155, 76740, 76843, 77294, 77559, 77687, 78077, 85450, 85482, 85838, 86153, 86379, 86450, 87414, 88771, 89603, 89689, 89793, 89869, 90713, 90856, 92337, 93171, 93668, 93777, 94056, 96161, 99902, 100029, 100762, 101258, 101268, 101984, 102293, 102395, 102609, 102744, 103135, 104030, 104511, 104567, 106341, 106504, 106685, 106950, 107612, 109723, 109816, 109917, 110200, 110426, 113617, 114437, 117194, 118760, 119088, 119227, 119278, 119794, 120604, 130414, 131566, 158131, 161292, 164538, 165196, 169590, 169858, 175536, 177262, 181627, 185101, 189456, 192455, 195909, 197569, 200472, 201290, 202381, 202711, 212830, 213682, 221581, 230025, 234837, 235198, 236285, 242888, 243575, 243931, 244092, 265651, 266425, 269389, 280462, 283283, 284457, 285906, 290661, 312700, 316768, 320193, 323332, 324264, 332831, 338450, 339135, 345549, 363143, 372824, 376890, 381936, 385017, 396401, 405821, 44954, 157044, 119263, 345032, 373416, 76104, 90310, 98112, 73107, 45557, 43255, 41085, 66227, 73705, 26421, 35151, 343095, 353969, 372532, 164993, 79844, 68278, 390336, 104706, 94979, 363290, 252963, 35244, 98559, 418753, 327210, 362165, 424227, 284674, 291213, 34303, 288330, 109661, 31334, 65513, 65651, 67500, 97115, 296711, 120626, 25919, 63035, 38477, 35942, 378947, 307639, 80157, 78056, 39937, 107623, 26955, 368711, 55618, 91027, 36431, 337881, 106438, 64866, 71502, 75165, 33238, 95341, 66904, 86136, 56111, 319970, 287138, 350232, 155796, 37465, 72926, 384806, 277909, 401504, 41088, 384369, 355857, 361498, 63141, 40765, 64541, 36022, 19777, 395173, 397619, 365109, 337721, 29971, 35664, 344604, 436727, 338095, 372784, 382073, 28401, 78133, 26129, 106613, 108410, 30696, 32284, 325596, 103838, 122768, 37611, 66154, 435100, 34922, 60453, 116791, 412019, 45186, 43362, 43560, 368774, 56579, 67900, 89839, 32404, 332381, 405422, 51899, 403016, 35798, 34521, 401244, 291928, 29047, 103239, 331338, 380599, 339642, 406158, 366780, 36191, 95467, 367555, 92240, 363095, 367089, 369053, 53001, 443567, 408961, 386140, 63678, 436058, 25862, 425661, 118635, 228277, 426155, 371606, 53677, 62037, 58155, 200768, 389957, 25929, 45883, 65143, 58409, 77474, 69303, 337868, 61847, 50188, 247303, 417395, 67535, 103704, 60474, 34277, 46000, 451850, 26266, 90163, 96769, 370754, 73293, 88658, 338075, 284929, 414852, 325007, 51713, 58642, 86551, 59314, 61695, 48682, 240200, 20686, 416496, 63661, 43633, 43030, 39437, 414982, 57168, 19585, 425598, 421994, 119901, 455857, 763304, 397078, 424942, 384814, 64473, 53750, 33107, 405094, 61537, 93435, 69019, 81414, 75932, 75790, 225481, 383353, 466856, 490196, 54777, 77889, 453494, 72402, 61473, 93743, 77372, 464105, 364955, 419256, 117715, 37343, 98333, 404802, 410400, 60097, 205725, 422720, 326429, 30848, 372122, 478209, 105187, 93873, 40338, 39313, 449467, 430674, 32155, 492506, 25028, 68379, 29030, 35703, 418832, 49743, 29747, 473753, 38577, 29923, 51433, 48037, 32080, 69289, 32342, 120088, 430779, 382383, 107152, 457430, 462519, 30746, 44040, 32412, 419946, 30418, 478049, 489270, 44170, 385057, 53454, 488539, 40367, 33740, 21148, 365830, 450259, 59710, 22550, 91786, 75177, 808185, 220806, 808146, 38984, 415234, 373445, 366721, 369359, 758743, 418004, 60168, 423505, 56210, 84861, 118414, 469937, 60586, 67229, 60287, 72317, 36104, 112681, 67647, 92226, 120186, 36154, 88326, 81036, 33253, 475937, 45943, 468492, 101725, 822854, 417614, 117214, 33802, 408318, 479537, 114863, 493247, 457433, 66491, 780595, 794374, 64482, 39840, 427089, 329737, 436460, 471030, 409904, 805185, 36807, 32981, 32264, 405163, 31235, 414931, 49531, 46511, 217894, 100851, 109195, 498311, 796314, 77904, 796335, 499556, 923752, 421143, 478134, 405508, 23940, 476643, 79182, 480269, 857265, 831884, 452623, 11565, 99053, 16473, 21505, 34522, 888693, 960890, 42274, 74312, 51047, 240684, 949489, 783608, 382053, 452608, 42692, 992911, 1282045, 71521, 45848, 870204, 782867, 68924, 1084950, 488905, 1063669, 426462, 1152282, 52270, 38033, 1183732, 850669, 847050, 54866, 17918, 118840, 27521, 233044, 1042877, 57935, 46384, 30254, 929860, 1205489, 870111, 24262, 34465, 52794, 43476, 1028539, 320668, 1034303, 38462, 64379, 59565, 101326, 44573, 92732, 49286, 40082, 68912, 39029, 62742, 879801, 51393, 46791, 38395, 69951, 1194271, 22698, 1287845, 44683, 23590, 24902, 27623, 50652, 454839, 37761, 46878, 26614, 121297, 1241195, 26273, 24500, 448179, 24516, 58323, 943512, 1020885, 43781, 62228, 73198, 35996, 56910, 1049413, 415679, 25455, 460742, 78128, 108596, 69976, 60358, 51362, 42906, 45886, 101874, 1144804, 242572, 35195, 73000, 110490, 76709, 100624, 68883, 472118, 70868, 90753, 35916, 41994, 315459, 25903, 25969, 25830, 923811, 77941, 112869, 407008, 96908, 1142972, 44186, 69446, 1363127, 48387, 1142433, 92550, 117201, 89066, 52609, 110936, 45124, 1423592, 45034, 73915, 1010055, 42179, 86542, 72761, 1093370, 25610, 103190, 34175, 61465, 73166, 94750, 49843, 63084, 56371, 99949, 100879, 106661, 60048, 1228953, 79839, 52954, 36203, 1425244, 42006, 298296, 435679, 87866, 76297, 94349, 138645, 9932, 91415, 110805, 40607, 114432, 119872, 22017, 19574, 93133, 242285, 35105, 499375, 1407050, 139153, 986213, 70077, 1144866, 214871, 847167, 418239, 147651, 61736, 258761, 271543, 806027, 28355, 51065, 68688, 85694, 35823, 1187043, 39931, 98692, 33226, 62041, 65872, 104892, 78382, 264695, 281173, 819755, 270645, 310851, 1140941, 1029241, 33175, 53772, 32636, 57318, 60879, 69421, 279695, 66482, 806147, 63633, 227194, 21873, 23241, 903622, 64828, 51786, 884762, 63094, 33987, 445953, 78269, 1198408, 33787, 32842, 41746, 90206, 68457, 892074, 1249294, 27255, 48191, 94642, 57933, 45070, 312859, 82479, 47840, 863136, 98197, 76507, 104008, 89366, 65233, 82088, 68286, 21549, 43044, 1391544, 59555, 191173, 62728, 95530, 30865, 59729, 60165, 42039, 62277, 44811, 40745, 1308667, 202989, 36367, 111002, 50271, 21874, 220016, 47101, 107175, 51706, 43599, 34902, 18526, 45333, 46076, 843302, 154129, 1183276, 101664, 45758, 79432, 107086, 41947, 41515, 50569, 61389, 38104, 34384, 79482, 1020543, 47550, 1511400, 396630, 71458, 32871, 54102, 62827, 37931, 93791, 60099, 43123, 58745, 23825, 88241, 1528718, 12651, 23911, 70287, 940580, 37980, 51534, 33110, 56341, 78878, 37702, 41373, 53729, 55073, 384699, 20876, 59829, 74720, 47167, 58124, 21106, 89821, 55571, 39080, 54164, 50699, 62893, 36748, 33389, 48210, 30396, 26932, 61597, 68675, 66318, 59447, 74400, 93593, 49177, 11541, 11071, 79802, 32946, 43458, 12255, 12224, 44905, 121411, 13025, 31002, 35157, 110076, 56308, 31968, 44888, 66492, 62203, 13422, 54446, 69080, 32410, 45029, 108583, 43938, 860906, 39502, 54653, 37420, 1112756, 46446, 80842, 184115, 214529, 21910, 111038, 29577, 31033, 465326, 54172, 27902, 53003, 41672, 86484, 882978, 50915, 34923, 68537, 71680, 35659, 56279, 33314, 40848, 72410, 373960, 22854, 76319, 23488, 43709, 372303, 1156506, 1331095, 1334585, 47279, 38238, 49881, 69745, 20570, 46750, 39895, 51667, 36038, 41530, 21750, 1020972, 68458, 50156, 1438535, 40000, 53752, 25791, 49367, 82243, 292081, 35447, 918575, 40444, 367204, 26264, 57400, 109982, 73190, 45049, 30265, 329679, 416773, 58166, 1255951, 34251, 19195, 45102, 79756, 54817, 104693, 100791, 37615, 100274, 71797, 97332, 32477, 34919, 103800, 472181, 64041, 419641, 56566, 106684, 70748, 14497, 17449, 45907, 72149, 66939, 44941, 19735, 53912, 25718, 109962, 38668, 851530, 1104006, 32447, 83713, 23326, 50135, 36840, 84867, 37054, 999913, 36919, 32610, 1825918, 112710, 88454, 37871, 28668, 73634, 51915, 99927, 28103, 31311, 43053, 46008, 207195, 66040, 207653, 37563, 52997, 38816, 51114, 449144, 163559, 1672723, 1488164, 294279, 42513, 77327, 50805, 36716, 112454, 685628, 42247, 67119, 48806, 48130, 23225, 441263, 984155, 126004, 1754141, 86610, 46432, 1461418, 96842, 109162, 84701, 36628, 28331, 154467, 61024, 71788, 44370, 42665, 34269, 65895, 251282, 64356, 140336, 33727, 69989, 118915, 30996, 57069, 67467, 77751, 482857, 52957, 21375, 371853, 39743, 76107, 39250, 58266, 288441, 1684927, 68967, 52188, 41415, 6826, 1507566, 73096, 160801, 32028, 37769, 58318, 100409, 383678, 7145, 58050, 7880, 61634, 1366409, 29508, 97951, 82883, 70898, 1640570, 68899, 111549, 122111, 49211, 37946, 85635, 59758, 55927, 177707, 24238, 72204, 54116, 46728, 64247, 27221, 455565, 99875, 395479, 89851, 93832, 32457, 131409, 1091172, 109770, 120428, 1767319, 38478, 27126, 279919, 116341, 36418, 65834, 31650, 114308, 87127, 43118, 176016, 29000, 25681, 352468, 281190, 53242, 54768, 54197, 31387, 109784, 41473, 174741, 113002, 276011, 1035729, 75922, 94826, 28872, 29571, 39054, 128137, 43150, 39288, 30657, 79002, 80623, 82106, 85864, 1839591, 75939, 39472, 33780, 37848, 79891, 86301, 97775, 1241017, 68294, 63013, 97550, 113376, 1577883, 34091, 101256, 49224, 179841, 91621, 53458, 33288, 34514, 71357, 102394, 122565, 64179, 267989, 74356, 42650, 64038, 2344672, 44110, 56089, 96440, 65400, 122685, 2655938, 51850, 449077, 228456, 210628, 98048, 51182, 99151, 20062, 36423, 87001, 2316787, 222012, 85363, 60440, 1697064, 57242, 2093099, 111552, 48289, 65380, 2140203, 46834, 65702, 96243, 152183, 76042, 73471, 56040, 69840, 179183, 104467, 95483, 32030, 70233, 7264, 53399, 109167, 72867, 60112, 2284766, 60307, 1341764, 1978524, 54601, 93970, 82592, 67622, 1690953, 262571, 218127, 43276, 38298, 806940, 1060256, 58917, 109761, 60291, 61411, 58615, 62150, 51690, 50485, 2292959, 34458, 45077, 240380, 44453, 50432, 31143, 28850, 60126, 66735, 68455, 69825, 57583, 74533, 113926, 67004, 282659, 48976, 181602, 45825, 100031, 1206543, 93238, 65198, 1040007, 50722, 94651, 86340, 2084773, 53272, 64684, 86896, 80616, 60558, 1373149, 69371, 166960, 1043694, 100888, 77247, 36395, 33366, 59205, 70122, 92676, 2343617, 65776, 1319722, 88765, 109323, 197273, 106479, 83900, 1029167, 1571565, 47400, 98965, 109498, 785012, 290014, 191019, 347330, 71679, 53651, 54881, 62979, 51015, 860907, 120381, 89488, 2421662, 94962, 340380, 76313, 91560, 60434, 44477, 3216510, 106683, 64688, 65744, 57967, 57556, 66221, 77233, 87373, 80764, 3194532, 8950, 124621, 162503, 1176095, 76706, 82461, 109760, 271984, 39595, 22409, 25725, 1723642, 99557, 35807, 38780, 24360, 135520, 135673, 29695, 59958, 48814, 27908, 267659, 116446, 85627, 22119, 2063008, 90631, 44417, 73168, 88133, 421045, 328955, 997184, 36882, 61429, 52104, 135166, 3070936, 67801, 78238, 1552624, 80116, 53802, 3146360, 72201, 112534, 303732, 94094, 58981, 28646, 132893, 99172, 106475, 57960, 70788, 44013, 71610, 109098, 54701, 67377, 28665, 87263, 73357, 83506, 38593, 88402, 831387, 406280, 69986, 75993, 79631, 78034, 49870, 75848, 390109, 69086, 66900, 41144, 45544, 12645, 61927, 910852, 26891, 46782, 88816, 54367, 56350, 77769, 1276940, 38354, 55459, 79966, 41387, 82769, 29949, 2582802, 70346, 64310, 35700, 2016872, 57905, 32396, 54651, 51019, 70673, 98492, 52662, 56897, 19504, 21542, 36189, 3793686, 5339, 150021, 31680, 34622, 75801, 58622, 73903, 51844, 54753, 61140, 3330828, 79936, 38056, 34985, 115826, 70578, 56082, 35681, 39244, 57920, 44488, 50233, 79615, 72278, 43390, 116014, 208502, 52141, 265752, 147310, 106539, 67315, 902999, 33378, 50714, 36709, 1621429, 32263, 426911, 52639, 3104818, 61931, 35784, 31828, 159603, 166973, 76760, 178130, 97074, 276868, 62384, 4282, 150176, 28138, 29058, 25880, 29395, 31104, 23469, 55864, 99300, 1238834, 867149, 470993, 96056, 56964, 92802, 2093977, 40257, 38449, 35617, 17029, 1239462, 49515, 87864, 99978, 48640, 38452, 87129, 95900, 43153, 38458, 71270, 485863, 62363, 83931, 22735, 82084, 48139, 75666, 32281, 32643, 2195566, 983909, 70672, 883391, 65692, 84781, 2215395, 13386, 64269, 1194616, 50323, 52749, 41328, 377057, 302531, 51732, 23973, 51605, 50483, 1623660, 3919322, 2429278, 40335, 40761, 42257, 43721, 45197, 52780, 65243, 98415, 91428, 60143, 78288, 148615, 115899, 185640, 93964, 74083, 65693, 70242, 39031, 65185, 62712, 2392385, 65441, 82193, 88116, 69464, 38965, 1599351, 1018877, 69318, 58512, 75260, 77130, 90601, 196931, 1065106, 67359, 2072025, 97176, 37678, 439771, 137799, 48593, 1714196, 60893, 58751, 154587, 30554, 41882, 1946298, 108556, 31477, 255702, 59915, 85328, 62479, 115820, 140340, 64704, 72189, 73042, 120135, 157171, 43687, 62713, 494260, 23943, 24647, 48346, 37104, 1734203, 41523, 293509, 61433, 110706, 57212, 90073, 53354, 114137, 28873, 55534, 104469, 35612, 29087, 31575, 26966, 36581, 49280, 34734, 79840, 262259, 67245, 26972, 29516, 58238, 46865, 382856, 367245, 43910, 44075, 55383, 23322, 21060, 58585, 25833, 47199, 81457, 55779, 68414, 492460, 27428, 29664, 55453, 107392, 42674, 67983, 48822, 309986, 163375, 54244, 36533, 40379, 35796, 60395, 44750, 70158, 51759, 951333, 37791, 49621, 44015, 63236, 63098, 37428, 49834, 481583, 47501, 55151, 15310, 55458, 1234542, 119238, 49682, 20479, 48588, 43079, 22981, 50580, 68748, 40438, 33858, 24465, 76644, 64691, 3878542, 192802, 29626, 1342403, 23396, 93999, 47150, 50560, 50771, 40732, 96160, 56673, 59298, 38929, 23202, 25499, 42786, 49035, 52148, 40835, 46239, 61401, 50993, 61173, 249081, 462598, 73880, 34546, 47579, 66984, 26185, 26292, 150648, 80832, 387301, 45261, 92159, 45037, 71550, 105815, 26683, 58719, 28353, 104401, 104761, 1496792, 289782, 59550, 63764, 3158416, 1578269, 64165, 248110, 2004419, 1580427, 84887, 49450, 63570, 46352, 61646, 63568, 1420554, 2226495, 192578, 206588, 3124476, 68183, 87386, 91411, 52234, 36391, 89265, 105127, 54037, 106301, 308558, 76555, 76554, 80401, 1379228, 2132486, 73735, 33760, 93986, 159752, 86636, 76851, 67710, 40137, 367495, 71864, 119310, 75052, 70599, 76112, 67236, 74703, 72007, 64626, 144246, 228853, 104741, 68577, 54358, 34208, 50717, 42033, 252985, 74494, 98606, 2719094, 67418, 147374, 73845, 37024, 4377918, 78928, 45331, 54923, 226009, 33808, 47325, 287337, 202236, 75344, 66212, 33337, 65595, 1278060, 180872, 80802, 82090, 100100, 207377, 95183, 89565, 92822, 106902, 62732, 76670, 71628, 79757, 103790, 3496372, 825279, 384252, 137201, 429095, 427219, 1114698, 46722, 42344, 92863, 56957, 33667, 78294, 72651, 113196, 120417, 38172, 39306, 3280916, 79520, 91173, 87385, 90079, 91165, 93315, 94592, 92217, 319147, 75208, 98411, 92210, 91909, 124869, 95381, 94037, 181012, 2551492, 73601, 94941, 88255, 80809, 94946, 85547, 52680, 91682, 86050, 68207, 88775, 98557, 85546, 80465, 460557, 288808, 1692190, 59773, 65677, 47479, 168785, 81233, 30089, 59933, 297169, 2404217, 64895, 69385, 56661, 456144, 41460, 69001, 974662, 50087, 20590, 69944, 69156, 39820, 43142, 85486, 74473, 35713, 207415, 98072, 92238, 73340, 20693, 2182169, 35009, 3155242, 74570, 110500, 94015, 17813, 26739, 37404, 242519, 66216, 97818, 104870, 29952, 31124, 31125, 29951, 300470, 52343, 49973, 1881060, 62428, 79619, 59529, 67321, 2556308, 248185, 453729, 114234, 61434, 86343, 90200, 79362, 76853, 995031, 432047, 291376, 93645, 66914, 97364, 77917, 3817848, 780660, 459516, 13823, 210804, 101609, 37737, 48217, 62207, 61658, 79211, 52921, 76208, 84698, 91091, 193524, 117287, 20414, 984204, 3973410, 71258, 6517, 2660888, 75357, 85665, 248428, 68161, 2199711, 94374, 364647, 80971, 290820, 408730, 2815072, 53879, 49593, 111190, 114306, 336325, 954541, 1038072, 200755, 68657, 69235, 63288, 56883, 303929, 100265, 165498, 57897, 189072, 369903, 3025994, 4215766, 78848, 189071, 253658, 290057, 480461, 357139, 189070, 418141, 1777608, 2586070, 2162709, 39711, 24660, 808508, 94813, 200208, 41423, 43325, 83102, 455135, 107606, 858437, 114376, 39583, 2664080, 79135, 88200, 303151, 487176, 1663673, 125015, 61504, 61719, 890885, 91244, 78963, 2704578, 41858, 51349, 411377, 100633, 163318, 108480, 294264, 66886, 102693, 120353, 29899, 75347, 169543, 94087, 17103, 108339, 55950, 100324, 324071, 217086, 1798148, 107259, 107260, 113273, 1704731, 2318158, 2356302, 3267334, 3417870, 2877108, 458050, 2152896, 4328798, 1483820, 76780, 2940482, 143213, 1198333, 1501298, 50085, 288324, 938706, 202055, 68562, 75620, 61203, 2542502, 95134, 65904, 64563, 101561, 37756, 246643, 1336601, 147995, 113330, 46789, 46280, 56472, 50299, 80966, 29542, 49933, 58241, 64155, 77587, 55423, 57851, 46937, 143296, 93910, 91213, 1350484, 87589, 173906, 79111, 455829, 282521, 345177, 117096, 28953, 24373, 124349, 847526, 98026, 114648, 77973, 62113, 1730695, 218616, 38687, 36301, 37034, 82754, 4180576, 4429128, 264734, 95511, 1714843, 59116, 45646, 66847, 4873, 56072, 36255, 38243, 810444, 69375, 115638, 80379, 63159, 40971, 2429414, 1413527, 23335, 16276, 3668162, 242256, 1954206, 1180583, 77834, 29819, 56467, 410696, 272360, 1233487, 442781, 24359, 57521, 202415, 117927, 75838, 1079968, 71782, 84136, 60325, 53876, 120356, 197857, 74487, 66075, 83677, 60737, 104575, 59437, 482374, 29611, 81053, 3817962, 155671, 83936, 433328, 415689, 2825120, 1156479, 410537, 398563, 455195, 39460, 6689, 7507, 97410, 7162, 97971, 58660, 45072, 87742, 880504, 71535, 4025514, 25211, 24792, 23215, 22647, 22899, 28368, 27281, 27100, 24285, 65470, 221905, 37414, 90788, 69141, 78976, 365043, 57535, 90903, 92516, 63979, 55106, 86289, 59589, 3800796, 4176604, 83805, 123102, 91328, 32686, 3741316, 117437, 233422, 118931, 152836, 218533, 67197, 98732, 375571, 60450, 79200, 99377, 61626, 97038, 240659, 57422, 78528, 81165, 62859, 291855, 70687, 84452, 44409, 217590, 37313, 3646462, 76513, 16310, 228021, 63272, 42697, 30277, 29754, 3168808, 79834, 40246, 85755, 3504542, 37960, 33558, 2075247, 1758563, 20987, 37823, 67511, 200052, 495241, 3591436, 110573, 99430, 127624, 90982, 319736, 39216, 76593, 76052, 1680102, 92144, 84845, 35189, 93312, 138074, 159568, 1032764, 1330015, 39320, 2303112, 31058, 2040537, 54756, 63381, 3094914, 83261, 40097, 826039, 60564, 114202, 80546, 114577, 39676, 104119, 84096, 70327, 2048865, 47978, 55119, 469318, 50692, 2444006, 2842470, 3141810, 1851988, 9369, 30781, 55839, 27688, 73589, 423372, 2964120, 2377398, 203672, 76902, 1977002, 76149, 62636, 114127, 68280, 124437, 1437361, 278291, 66963, 104298, 315632, 294662, 53806, 370295, 104787, 339918, 1278153, 62493, 57558, 58920, 82813, 65877, 362771, 366985, 426075, 396563, 451833, 349878, 454431, 124811, 416712, 72648, 1110059, 59655, 34076, 392364, 60787, 69690, 118717, 1786650, 64729, 70783, 62997, 99384, 267675, 50987, 47962, 88359, 488798, 99406, 108342, 858492, 1105709, 1003005, 133146, 63544, 56703, 288276, 269138, 54827, 293849, 1630603, 126562, 337717, 61070, 180316, 326976, 491661, 59324, 175743, 83634, 41472, 58523, 384309, 4865436, 2190193, 1891788, 101477, 92720, 65378, 329083, 309047, 1609168, 105271, 1355546, 1922545, 422908, 74705, 28808, 32262, 5491920, 51927, 220640, 182730, 164063, 44406, 100581, 79191, 85375, 44838, 72882, 435638, 100950, 54628, 31012, 78073, 42266, 43356, 43432, 44700, 45633, 46011, 49101, 58631, 76516, 47121, 38908, 119415, 1821317, 4991384, 62693, 31000, 3662, 51214, 3828796, 104129, 92669, 82338, 33403, 783521, 2395421, 3906082, 85863, 5200962, 3715854, 52844, 61067, 920457, 2555958, 4258292, 1705772, 5178278, 4987556, 4806232, 5128328, 4937812, 3320578, 1857670, 99043, 368400, 417241, 807944, 1039952, 984177, 1388441, 377321, 984202, 284971, 152979, 161981, 205451, 4188654, 57380, 70588, 463028, 79655, 27684, 103008, 74187, 1891755, 449951, 1655607, 376144, 375878, 1725795, 464071, 376127, 376076, 335185, 80485, 72012, 77278, 1252596, 65788, 52227, 66920, 58533, 67303, 388154, 2838678, 1325734, 4016934, 2391833, 1857772, 2577666, 1235534, 1849868, 84408, 135534, 46333, 2072214, 154684, 80648, 63531, 79882, 235686, 1667418, 56110, 104874, 98972, 33853, 795176, 68662, 382205, 92855, 40861, 39490, 76440, 46714, 78978, 50545, 51507, 80321, 28726, 86476, 1910605, 2263814, 65407, 117017, 124014, 433367, 5613056, 38611, 40481, 31517, 39861, 175471, 53214, 5275830, 5503594, 41771, 5065810, 99595, 81421, 65750, 67397, 92297, 62065, 20891, 100273, 85175, 24768, 66256, 92967, 1517471, 1153101, 451214, 3598648, 1209380, 33257, 5071886, 127933, 83657, 97136, 252133, 329913, 128725, 357282, 57892, 190583, 66268, 255851, 111081, 82880, 102619, 66258, 365880, 47464, 49125, 68622, 27133, 47642, 52389, 100389, 56640, 2166214, 290937, 377071, 3071532, 78320, 219105, 3718778, 42514, 58101, 76582, 111199, 5692030, 2596918, 491603, 3000844, 3922798, 54826, 67576, 93991, 88757, 83168]

# Drop rows where "imdbId" matches any value in failed_imdbIds
df = df[~df['imdbId'].isin(failed_downloads)]

In [18]:
df.shape

(32899, 18)

In [19]:
df.to_csv("15_clean_MovieGenre.csv", index=False)

In [20]:
genre_counts = df.iloc[:, 3:].sum(axis=0)
genre_counts = genre_counts.sort_values(ascending=False)

# Print the sorted sum for each binarized genre
print(genre_counts)
print(df.shape)

Drama          16769
Comedy         10627
Romance         5051
Action          4488
Crime           4319
Thriller        4070
Horror          3361
Documentary     3248
Adventure       3185
Mystery         1973
Family          1762
Sci-Fi          1708
Fantasy         1694
Biography       1685
Animation       1457
dtype: int64
(32899, 18)
